How the simulation is done:
- First, a base simulation is done per sample size.
- Then, as needed, more simulations are done by upticking the iteration values.

Here, we only compute the MSEs for the intended sample, which results in a faster simulation than before

# Imports:

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import statsmodels.api as sm

import sys
sys.path.append('../')
from ddc_utils import is_binomial_data_seperable
from jzhou_utils import save_obj_pickle

# Hyperparams:

In [4]:
pop_index = 1
iter_val = 11

In [5]:
rand_generator = np.random.default_rng(seed=333 * pop_index + iter_val)

In [6]:
population_size = 100_000
number_of_coefficients = 1

num_iters_per_population_for_small_samples = 50_000
num_iters_per_population_for_large_samples = 1_000
small_large_sample_co = 100

# biased sampling scheme params:
sample_probability_centering = 0.77
sample_probability_bias_factor = 1

In [7]:
ALL_SAMPLE_SIZES = (
    [i for i in range(3, 15)]
)

In [8]:
len(ALL_SAMPLE_SIZES)

12

# Run:

## Load Finite Population Data:

In [9]:
pickle_filename = f'base_population_data_Logit_1.pickle'
pop_data = pd.read_pickle(pickle_filename)

## get population-level statistics:

In [ ]:
feature_cols = [f"x_{i}" for i in range(number_of_coefficients)]
pop_x = pop_data[feature_cols]
pop_y = pop_data["y"]

pop_model = sm.Logit(endog=pop_y, exog=pop_x).fit(disp=0)
pop_beta = np.array(pop_model.params)
pop_gs = pop_x * (
    np.array(pop_y).reshape((population_size, 1))
    - np.array(pop_model.predict()).reshape((population_size, 1))
)

## iterate:

In [13]:
mse_means = {}
mse_stdevs = {}

In [14]:
for temp_sample_size in tqdm(ALL_SAMPLE_SIZES):
    print(temp_sample_size)
    # where the data will be saved:
    all_sample_beta_full = []
    all_realized_sample_sizes = []

    # set up how much to sample for this population:
    non_separable_count = 0
    if temp_sample_size < small_large_sample_co:
        num_iters_per_population = num_iters_per_population_for_small_samples
    else:
        num_iters_per_population = num_iters_per_population_for_large_samples

    for _ in trange(num_iters_per_population, mininterval=10):
        """
        First, sample a valid dataset (which is not seperable):
        """
        # use sampling scheme to sample data:
        obtained_valid_sample = False

        while not obtained_valid_sample:
            # intended sample:
            pop_data["r0"] = 0
            pop_data.loc[
                np.random.choice(pop_data.index, size=temp_sample_size, replace=False),
                "r0",
            ] = 1

            full_sampled_data = pop_data[pop_data["r0"] == 1]

            # if the sample size is too small, check for seperability:
            if temp_sample_size < 1_000:
                if is_binomial_data_seperable(full_sampled_data, "y", "x_0"):
                    non_separable_count = non_separable_count + 1
                    continue

            obtained_valid_sample = True

        """
            Then, compute the logistic MSEs only:
        """
        # compute full x, y, model, beta
        sample_x_full, sample_y_full = (
            full_sampled_data[feature_cols],
            full_sampled_data["y"],
        )
        sample_beta_full = np.array(
            sm.Logit(endog=sample_y_full, exog=sample_x_full)
            .fit(disp=0, maxiter=5_00)
            .params
        )

        # compute full versions of things:
        all_sample_beta_full.append(pd.Series(sample_beta_full))

    all_mses = (pd.concat(all_sample_beta_full).reset_index(drop=True) - pop_beta) ** 2
    mse_means[temp_sample_size] = np.mean(all_mses)
    mse_stdevs[temp_sample_size] = np.std(all_mses) / np.sqrt(num_iters_per_population)

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

3



%|                                                                                        | 0/50000 [00:00<?, ?it/s]
%|█▏                                                                            | 783/50000 [00:10<10:28, 78.25it/s]
%|█▏                                                                            | 783/50000 [00:20<10:28, 78.25it/s]
%|██▍                                                                          | 1587/50000 [00:20<10:09, 79.39it/s]
%|███▋                                                                         | 2389/50000 [00:30<10:08, 78.26it/s]
%|████▉                                                                        | 3168/50000 [00:40<09:59, 78.10it/s]
%|██████                                                                       | 3947/50000 [00:50<09:51, 77.81it/s]
%|███████▎                                                                     | 4720/50000 [01:00<09:51, 76.55it/s]
%|████████▌                                                    

4



%|                                                                                        | 0/50000 [00:00<?, ?it/s]
%|██▏                                                                         | 1444/50000 [00:10<05:36, 144.31it/s]
%|████▍                                                                       | 2899/50000 [00:20<05:24, 144.97it/s]
%|██████▋                                                                     | 4372/50000 [00:30<05:12, 146.01it/s]
%|████████▉                                                                   | 5846/50000 [00:40<05:01, 146.54it/s]
%|███████████▏                                                                | 7333/50000 [00:50<04:49, 147.31it/s]
%|█████████████▍                                                              | 8820/50000 [01:00<04:45, 144.43it/s]
%|███████████████▍                                                           | 10286/50000 [01:10<04:33, 145.12it/s]
%|█████████████████▋                                           

...

14



%|                                                                                        | 0/50000 [00:00<?, ?it/s]
%|████▊                                                                       | 3170/50000 [00:10<02:27, 316.95it/s]
%|█████████▋                                                                  | 6340/50000 [00:20<02:18, 315.45it/s]
%|██████████████▍                                                             | 9496/50000 [00:30<02:08, 315.50it/s]
%|██████████████████▉                                                        | 12652/50000 [00:40<01:58, 314.17it/s]
%|███████████████████████▋                                                   | 15774/50000 [00:50<01:49, 311.74it/s]
%|████████████████████████████▎                                              | 18863/50000 [01:00<01:40, 310.78it/s]
%|████████████████████████████████▉                                          | 21952/50000 [01:10<01:30, 308.57it/s]
%|█████████████████████████████████████▌                       

 21%|████████████████▍                                                            | 2136/10000 [02:53<10:46, 12.17it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 23%|█████████████████▍                                                           | 2263/10000 [03:03<10:29, 12.30it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 24%|██████████████████▍                                                          | 2389/10000 [03:15<10:48, 11.74it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: 

 46%|███████████████████████████████████▍                                         | 4606/10000 [06:31<08:02, 11.19it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 48%|█████████████████████████████████████▏                                       | 4828/10000 [06:51<07:51, 10.98it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 49%|█████████████████████████████████████▉                                       | 4935/10000 [07:01<07:45, 10.87it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: 


 75%|█████████████████████████████████████████████████████████▊                   | 7510/10000 [11:00<03:41, 11.23it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)

 78%|████████████████████████████████████████████████████████████▍                | 7843/10000 [11:31<03:17, 10.93it/s]C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.conda\envs\airsoft\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Jason\.

## merge data, save:

In [ ]:
mse_means = pd.DataFrame.from_dict(mse_means, orient="index")
mse_means.columns = ["mean"]

mse_stdevs = pd.DataFrame.from_dict(mse_stdevs, orient="index")
mse_stdevs.columns = ["stdev"]

mse_data = mse_means.join(mse_stdevs)
mse_data["count"] = np.where(
    mse_data.index < small_large_sample_co,
    num_iters_per_population_for_small_samples,
    num_iters_per_population_for_large_samples,
)

save_obj_pickle(f"srs_sim_results/sim_srs_iter_{iter_val}.pickle", mse_data)